# LSTM Model for Fake News Detection

Este notebook implementa uma rede neural Long Short-Term Memory (LSTM) para deteção de notícias falsas. As redes LSTM são particularmente eficazes para dados sequenciais como texto, pois conseguem capturar dependências de longo alcance e informação contextual.

## Porquê LSTM para Deteção de Notícias Falsas?

As redes LSTM oferecem várias vantagens para tarefas de classificação de texto:
1. Capturam padrões sequenciais e dependências no texto
2. Mantêm memória de partes anteriores do texto ao processar partes posteriores
3. Conseguem aprender padrões linguísticos complexos que podem indicar deceção

## Considerações para Databricks Community Edition

**IMPORTANTE**: Este notebook inclui estratégias para lidar com as limitações da Databricks Community Edition:
- Memória limitada (15.3 GB)
- Apenas 2 cores de processamento
- Tempo de execução limitado (desligamento após período de inatividade)

Dependendo dos recursos disponíveis, pode optar por:
1. **Dataset completo**: Para resultados ótimos (requer mais recursos)
2. **Amostragem estratificada**: Para desenvolvimento e testes rápidos
3. **Validação cruzada**: Para avaliação robusta mesmo com amostras menores

## Setup e Imports

Primeiro, vamos importar as bibliotecas necessárias e configurar o ambiente.

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import time
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from pyspark.sql import SparkSession, DataFrame
from pyspark.sql.functions import col, udf, lit
from pyspark.sql.types import StringType, ArrayType

## Criar Estrutura de Diretórios

Configurar os diretórios necessários para armazenar modelos, logs e resultados.

In [ ]:
def create_directory_structure():
    """Criar estrutura de diretórios para modelos e logs no DBFS.
    
    Esta função cria os diretórios necessários para armazenar:
    - Modelos: Modelos LSTM treinados e tokenizadores
    - Logs: Métricas de desempenho e visualizações
    - Validação cruzada: Resultados de cada fold
    """
    # No Databricks, usamos dbutils para interagir com o DBFS
    directories = [
        "dbfs:/FileStore/fake_news_detection/models/lstm",
        "dbfs:/FileStore/fake_news_detection/logs",
        "dbfs:/FileStore/fake_news_detection/models/lstm/cv_folds"
    ]
    
    for directory in directories:
        # Remover prefixo dbfs: para dbutils.fs.mkdirs
        dir_path = directory.replace("dbfs:", "")
        dbutils.fs.mkdirs(dir_path)
        print(f"Diretório criado: {directory}")

## Inicializar Sessão Spark

Criar uma sessão Spark com configuração apropriada para a Databricks Community Edition.

In [ ]:
def initialize_spark():
    """Inicializar uma sessão Spark com configuração otimizada para Databricks Community Edition."""
    spark = SparkSession.builder \
        .appName("FakeNewsLSTM") \
        .config("spark.sql.shuffle.partitions", "8") \
        .config("spark.driver.memory", "8g") \
        .enableHiveSupport() \
        .getOrCreate()
    
    print(f"Versão do Spark: {spark.version}")
    return spark

## Carregar e Preparar Dados

Carregar os dados pré-processados da etapa de engenharia de características. Para o treino do LSTM, oferecemos opções para usar o dataset completo ou uma amostra estratificada, dependendo dos recursos disponíveis.

In [ ]:
def load_data(spark, use_sample=False, sample_size_per_class=None):
    """Carregar dados pré-processados da etapa de engenharia de características.
    
    Args:
        spark: Objeto SparkSession
        use_sample: Se True, usa amostragem estratificada para treino com recursos limitados
        sample_size_per_class: Número de registros por classe para a amostra (se None e use_sample=True, usa 5000)
        
    Returns:
        DataFrame: Spark DataFrame contendo os dados
    """
    # Definir caminhos
    feature_data_path = "dbfs:/FileStore/fake_news_detection/data/feature_data/features.parquet"
    processed_data_path = "dbfs:/FileStore/fake_news_detection/data/processed_data/processed_news.parquet"
    
    print("Carregando dados...")
    try:
        # Tentar carregar primeiro da saída de engenharia de características
        df = spark.read.parquet(feature_data_path)
        print(f"Dados com características carregados de {feature_data_path}")
    except Exception as e:
        print(f"Não foi possível carregar dados com características: {e}")
        # Recorrer a dados processados
        try:
            df = spark.read.parquet(processed_data_path)
            print(f"Dados processados carregados de {processed_data_path}")
        except Exception as e2:
            print(f"Não foi possível carregar dados processados: {e2}")
            # Último recurso: tentar tabelas Hive
            try:
                df = spark.table("processed_news")
                print("Dados carregados da tabela Hive 'processed_news'")
            except Exception as e3:
                print(f"Erro ao carregar dados de todas as fontes: {e3}")
                print("Saindo pois nenhuma fonte de dados está disponível.")
                return None
    
    # Imprimir informações do dataset
    total_count = df.count()
    print(f"Total de registros: {total_count}")
    print("Distribuição de classes:")
    class_counts = df.groupBy("label").count()
    class_counts.show()
    
    # Verificar recursos disponíveis
    print("\nVerificando recursos disponíveis...")
    import psutil
    available_memory_gb = psutil.virtual_memory().available / (1024 ** 3)
    print(f"Memória disponível: {available_memory_gb:.2f} GB")
    
    # Criar amostra estratificada se solicitado ou se o dataset for muito grande para os recursos disponíveis
    if use_sample:
        if sample_size_per_class is None:
            # Valor padrão para amostra na Community Edition
            sample_size_per_class = 5000
            
        print(f"\nCriando amostra estratificada com {sample_size_per_class} registros por classe...")
        
        # Obter contagens por classe
        class_counts_dict = {row['label']: row['count'] for row in class_counts.collect()}
        
        # Calcular frações para amostragem estratificada
        fractions = {}
        for label, count in class_counts_dict.items():
            fractions[label] = min(1.0, sample_size_per_class / count)
        
        # Criar amostra estratificada
        df = df.sampleBy("label", fractions=fractions, seed=42)
        
        print("Amostra criada. Nova distribuição de classes:")
        df.groupBy("label").count().show()
        
        print("\nNOTA: Usando amostra estratificada devido a limitações de recursos.")
        print("Para resultados ótimos, considere usar o dataset completo se tiver recursos suficientes.")
    else:
        # Verificar se o dataset completo é viável para os recursos disponíveis
        estimated_memory_needed_gb = total_count * 0.001  # Estimativa grosseira: ~1MB por registro
        if estimated_memory_needed_gb > available_memory_gb * 0.7:  # Usar no máximo 70% da memória disponível
            print("\nAVISO: O dataset completo pode exceder a memória disponível.")
            print("Considere usar amostragem estratificada (use_sample=True) para evitar problemas de memória.")
            print("Continuando com o dataset completo, mas monitore o uso de memória.")
    
    return df

## Preparar Dados de Texto para LSTM

Processar os dados de texto para um formato adequado para treino LSTM, incluindo tokenização e padding.

In [ ]:
def prepare_text_data(df, text_column="processed_text", max_words=20000, max_len=300):
    """Preparar dados de texto para modelo LSTM através de tokenização e padding.
    
    Args:
        df: DataFrame Pandas contendo os dados de texto
        text_column: Nome da coluna contendo o texto a processar
        max_words: Número máximo de palavras no vocabulário
        max_len: Comprimento máximo das sequências
        
    Returns:
        tuple: (X_train_pad, X_test_pad, y_train, y_test, tokenizer)
    """
    print("Preparando dados de texto para LSTM...")
    
    # Preencher valores NaN
    df[text_column] = df[text_column].fillna('')
    
    # Dividir dados com estratificação
    X_train, X_test, y_train, y_test = train_test_split(
        df[text_column], 
        df['label'], 
        test_size=0.2, 
        random_state=42, 
        stratify=df['label']
    )
    
    print(f"Tamanho do conjunto de treino: {len(X_train)}")
    print(f"Tamanho do conjunto de teste: {len(X_test)}")
    
    # Tokenizar texto
    print("Tokenizando texto...")
    tokenizer = Tokenizer(num_words=max_words)
    tokenizer.fit_on_texts(X_train)
    
    X_train_seq = tokenizer.texts_to_sequences(X_train)
    X_test_seq = tokenizer.texts_to_sequences(X_test)
    
    # Padding de sequências
    X_train_pad = pad_sequences(X_train_seq, maxlen=max_len)
    X_test_pad = pad_sequences(X_test_seq, maxlen=max_len)
    
    print(f"Tamanho do vocabulário: {min(len(tokenizer.word_index) + 1, max_words)}")
    print(f"Comprimento da sequência: {max_len}")
    
    return X_train_pad, X_test_pad, y_train, y_test, tokenizer

## Construir Modelo LSTM

Criar uma arquitetura de modelo LSTM bidirecional para classificação de texto, com opções para ajustar a complexidade baseada nos recursos disponíveis.

In [ ]:
def build_lstm_model(vocab_size, embedding_dim=100, max_len=300, complexity="standard"):
    """Construir um modelo LSTM bidirecional para classificação de texto.
    
    Args:
        vocab_size: Tamanho do vocabulário
        embedding_dim: Dimensão da camada de embedding
        max_len: Comprimento máximo das sequências de entrada
        complexity: Nível de complexidade do modelo ("light", "standard", "deep")
        
    Returns:
        model: Modelo Keras compilado
    """
    print(f"Construindo modelo LSTM bidirecional (complexidade: {complexity})...")
    
    model = Sequential()
    model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_len))
    
    if complexity == "light":  # Modelo leve para recursos limitados
        model.add(Bidirectional(LSTM(units=64, return_sequences=False)))
        model.add(Dropout(0.5))
        model.add(Dense(32, activation='relu'))
    
    elif complexity == "standard":  # Modelo padrão (bom equilíbrio)
        model.add(Bidirectional(LSTM(units=128, return_sequences=True)))
        model.add(Dropout(0.5))
        model.add(Bidirectional(LSTM(units=64, return_sequences=False)))
        model.add(Dropout(0.5))
        model.add(Dense(32, activation='relu'))
    
    else:  # Modelo profundo para recursos abundantes
        model.add(Bidirectional(LSTM(units=256, return_sequences=True)))
        model.add(Dropout(0.5))
        model.add(Bidirectional(LSTM(units=128, return_sequences=True)))
        model.add(Dropout(0.5))
        model.add(Bidirectional(LSTM(units=64, return_sequences=False)))
        model.add(Dropout(0.5))
        model.add(Dense(64, activation='relu'))
        model.add(Dense(32, activation='relu'))
    
    # Camada de saída (comum a todos os modelos)
    model.add(Dense(1, activation='sigmoid'))
    
    model.compile(
        optimizer='adam',
        loss='binary_crossentropy',
        metrics=['accuracy']
    )
    
    print(model.summary())
    return model

## Treinar com Validação Cruzada

Treinar o modelo LSTM usando validação cruzada k-fold para uma avaliação mais robusta, especialmente importante quando se trabalha com amostras.

In [ ]:
def train_with_cross_validation(X, y, vocab_size, embedding_dim=100, max_len=300, n_folds=5, complexity="standard", batch_size=64, epochs=10):
    """Treinar modelo LSTM com validação cruzada k-fold.
    
    Args:
        X: Sequências com padding
        y: Rótulos alvo
        vocab_size: Tamanho do vocabulário
        embedding_dim: Dimensão da camada de embedding
        max_len: Comprimento máximo das sequências de entrada
        n_folds: Número de folds para validação cruzada
        complexity: Nível de complexidade do modelo ("light", "standard", "deep")
        batch_size: Tamanho do batch para treino
        epochs: Número máximo de épocas para treino
        
    Returns:
        tuple: (models, histories, scores, metrics_df)
    """
    print(f"Treinando com validação cruzada {n_folds}-fold...")
    
    # Converter para arrays numpy se forem Series pandas
    if isinstance(y, pd.Series):
        y = y.values
    
    # Definir estratégia de validação cruzada
    kfold = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=42)
    
    # Listas para armazenar resultados
    models = []
    histories = []
    scores = []
    fold_metrics = []
    
    # Ajustar batch_size baseado no tamanho dos dados
    if len(X) < 1000:
        batch_size = min(batch_size, 32)  # Batch menor para datasets pequenos
        print(f"Ajustando batch_size para {batch_size} devido ao tamanho do dataset")
    
    # Loop de validação cruzada
    for fold, (train_idx, val_idx) in enumerate(kfold.split(X, y)):
        print(f"\nTreinando fold {fold+1}/{n_folds}")
        
        # Dividir dados
        X_train_fold, X_val_fold = X[train_idx], X[val_idx]
        y_train_fold, y_val_fold = y[train_idx], y[val_idx]
        
        # Construir modelo
        model = build_lstm_model(vocab_size, embedding_dim, max_len, complexity)
        
        # Definir callbacks
        early_stopping = EarlyStopping(
            monitor='val_loss',
            patience=3,
            restore_best_weights=True
        )
        
        checkpoint_path = f"/tmp/lstm_model_fold_{fold+1}.keras"
        model_checkpoint = ModelCheckpoint(
            checkpoint_path,
            monitor='val_loss',
            save_best_only=True,
            mode='min'
        )
        
        # Registrar tempo de início
        start_time = time.time()
        
        # Treinar modelo
        history = model.fit(
            X_train_fold, y_train_fold,
            epochs=epochs,
            batch_size=batch_size,
            validation_data=(X_val_fold, y_val_fold),
            callbacks=[early_stopping, model_checkpoint],
            verbose=1
        )
        
        # Calcular tempo de treino
        train_time = time.time() - start_time
        print(f"Tempo de treino: {train_time:.2f} segundos")
        
        # Avaliar modelo
        score = model.evaluate(X_val_fold, y_val_fold, verbose=0)
        print(f"Fold {fold+1} - Loss: {score[0]:.4f}, Accuracy: {score[1]:.4f}")
        
        # Previsões
        y_pred_proba = model.predict(X_val_fold)
        y_pred = (y_pred_proba > 0.5).astype(int).flatten()
        
        # Calcular métricas
        accuracy = accuracy_score(y_val_fold, y_pred)
        precision = precision_score(y_val_fold, y_pred, average='weighted')
        recall = recall_score(y_val_fold, y_pred, average='weighted')
        f1 = f1_score(y_val_fold, y_pred, average='weighted')
        
        fold_metrics.append({
            'fold': fold+1,
            'accuracy': accuracy,
            'precision': precision,
            'recall': recall,
            'f1': f1,
            'train_time': train_time
        })
        
        # Armazenar resultados
        models.append(model)
        histories.append(history)
        scores.append(score[1])  # Accuracy
        
        # Copiar modelo para DBFS
        dbutils.fs.cp(f"file:{checkpoint_path}", f"dbfs:/FileStore/fake_news_detection/models/lstm/cv_folds/lstm_model_fold_{fold+1}.keras")
        
        # Liberar memória
        tf.keras.backend.clear_session()
    
    # Imprimir desempenho médio
    print(f"\nValidação cruzada completa. Accuracy média: {np.mean(scores):.4f} ± {np.std(scores):.4f}")
    
    # Criar DataFrame com métricas por fold
    metrics_df = pd.DataFrame(fold_metrics)
    print("\nMétricas por fold:")
    print(metrics_df)
    
    # Salvar métricas em CSV
    metrics_csv_path = "/tmp/lstm_cv_metrics.csv"
    metrics_df.to_csv(metrics_csv_path, index=False)
    dbutils.fs.cp(f"file:{metrics_csv_path}", "dbfs:/FileStore/fake_news_detection/logs/lstm_cv_metrics.csv")
    
    return models, histories, scores, metrics_df

## Treinar Modelo Final

Treinar um modelo final no conjunto de treino completo após a validação cruzada.

In [ ]:
def train_final_model(X_train, y_train, X_test, y_test, vocab_size, embedding_dim=100, max_len=300, complexity="standard", batch_size=64, epochs=10):
    """Treinar modelo final no conjunto de treino completo após validação cruzada.
    
    Args:
        X_train: Sequências de treino com padding
        y_train: Rótulos de treino
        X_test: Sequências de teste com padding
        y_test: Rótulos de teste
        vocab_size: Tamanho do vocabulário
        embedding_dim: Dimensão da camada de embedding
        max_len: Comprimento máximo das sequências de entrada
        complexity: Nível de complexidade do modelo ("light", "standard", "deep")
        batch_size: Tamanho do batch para treino
        epochs: Número máximo de épocas para treino
        
    Returns:
        tuple: (model, history, test_metrics)
    """
    print("\nTreinando modelo final no conjunto de treino completo...")
    
    # Construir modelo
    model = build_lstm_model(vocab_size, embedding_dim, max_len, complexity)
    
    # Definir callbacks
    early_stopping = EarlyStopping(
        monitor='val_loss',
        patience=3,
        restore_best_weights=True
    )
    
    checkpoint_path = "/tmp/lstm_model_final.keras"
    model_checkpoint = ModelCheckpoint(
        checkpoint_path,
        monitor='val_loss',
        save_best_only=True,
        mode='min'
    )
    
    # Registrar tempo de início
    start_time = time.time()
    
    # Treinar modelo
    history = model.fit(
        X_train, y_train,
        epochs=epochs,
        batch_size=batch_size,
        validation_split=0.1,  # 10% para validação
        callbacks=[early_stopping, model_checkpoint],
        verbose=1
    )
    
    # Calcular tempo de treino
    train_time = time.time() - start_time
    print(f"Tempo de treino: {train_time:.2f} segundos")
    
    # Avaliar no conjunto de teste
    test_loss, test_acc = model.evaluate(X_test, y_test, verbose=1)
    print(f"Accuracy no teste: {test_acc:.4f}")
    
    # Previsões
    y_pred_proba = model.predict(X_test)
    y_pred = (y_pred_proba > 0.5).astype(int).flatten()
    
    # Calcular métricas
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='weighted')
    recall = recall_score(y_test, y_pred, average='weighted')
    f1 = f1_score(y_test, y_pred, average='weighted')
    
    # Relatório de classificação
    print("\nRelatório de classificação:")
    print(classification_report(y_test, y_pred))
    
    # Matriz de confusão
    cm = confusion_matrix(y_test, y_pred)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Fake', 'Real'], yticklabels=['Fake', 'Real'])
    plt.xlabel('Previsto')
    plt.ylabel('Real')
    plt.title('Matriz de Confusão')
    plt.tight_layout()
    
    # Salvar figura
    cm_path = "/tmp/confusion_matrix.png"
    plt.savefig(cm_path)
    dbutils.fs.cp(f"file:{cm_path}", "dbfs:/FileStore/fake_news_detection/logs/lstm_confusion_matrix.png")
    
    # Salvar modelo final
    dbutils.fs.cp(f"file:{checkpoint_path}", "dbfs:/FileStore/fake_news_detection/models/lstm/lstm_model_final.keras")
    
    # Métricas de teste
    test_metrics = {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1,
        'train_time': train_time
    }
    
    return model, history, test_metrics

## Visualizar Histórico de Treino

Visualizar o histórico de treino para analisar o desempenho do modelo ao longo do tempo.

In [ ]:
def plot_training_history(history):
    """Visualizar histórico de treino do modelo.
    
    Args:
        history: Objeto history retornado por model.fit()
    """
    # Accuracy
    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('Accuracy do Modelo')
    plt.ylabel('Accuracy')
    plt.xlabel('Época')
    plt.legend(['Treino', 'Validação'], loc='lower right')
    
    # Loss
    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('Loss do Modelo')
    plt.ylabel('Loss')
    plt.xlabel('Época')
    plt.legend(['Treino', 'Validação'], loc='upper right')
    
    plt.tight_layout()
    
    # Salvar figura
    history_path = "/tmp/training_history.png"
    plt.savefig(history_path)
    dbutils.fs.cp(f"file:{history_path}", "dbfs:/FileStore/fake_news_detection/logs/lstm_training_history.png")

## Salvar Tokenizer

Salvar o tokenizer para uso futuro na inferência.

In [ ]:
def save_tokenizer(tokenizer):
    """Salvar tokenizer para uso futuro na inferência.
    
    Args:
        tokenizer: Objeto tokenizer treinado
    """
    tokenizer_path = "/tmp/tokenizer.pickle"
    with open(tokenizer_path, 'wb') as handle:
        pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
    dbutils.fs.cp(f"file:{tokenizer_path}", "dbfs:/FileStore/fake_news_detection/models/lstm/tokenizer.pickle")
    print("Tokenizer salvo em dbfs:/FileStore/fake_news_detection/models/lstm/tokenizer.pickle")

## Função Principal

Função principal para executar o pipeline completo de treino LSTM.

In [ ]:
def main(use_sample=True, sample_size_per_class=5000, complexity="light", n_folds=3):
    """Executar pipeline completo de treino LSTM.
    
    Args:
        use_sample: Se True, usa amostragem estratificada para treino com recursos limitados
        sample_size_per_class: Número de registros por classe para a amostra
        complexity: Nível de complexidade do modelo ("light", "standard", "deep")
        n_folds: Número de folds para validação cruzada
    """
    print("Iniciando pipeline de treino LSTM para deteção de notícias falsas...")
    
    # Criar estrutura de diretórios
    create_directory_structure()
    
    # Inicializar Spark
    spark = initialize_spark()
    
    # Carregar dados
    df = load_data(spark, use_sample=use_sample, sample_size_per_class=sample_size_per_class)
    if df is None:
        return
    
    # Converter para Pandas para processamento
    print("\nConvertendo para DataFrame Pandas...")
    pandas_df = df.toPandas()
    
    # Preparar dados de texto
    X_train_pad, X_test_pad, y_train, y_test, tokenizer = prepare_text_data(pandas_df)
    
    # Tamanho do vocabulário
    vocab_size = min(len(tokenizer.word_index) + 1, 20000)
    
    # Treinar com validação cruzada
    models, histories, scores, metrics_df = train_with_cross_validation(
        X_train_pad, y_train, vocab_size, 
        n_folds=n_folds, 
        complexity=complexity,
        batch_size=32 if use_sample and sample_size_per_class < 1000 else 64
    )
    
    # Treinar modelo final
    final_model, final_history, test_metrics = train_final_model(
        X_train_pad, y_train, X_test_pad, y_test, vocab_size,
        complexity=complexity,
        batch_size=32 if use_sample and sample_size_per_class < 1000 else 64
    )
    
    # Visualizar histórico de treino
    plot_training_history(final_history)
    
    # Salvar tokenizer
    save_tokenizer(tokenizer)
    
    print("\nPipeline de treino LSTM concluído com sucesso!")
    print(f"Modelo final salvo em dbfs:/FileStore/fake_news_detection/models/lstm/lstm_model_final.keras")
    print(f"Métricas de teste: {test_metrics}")
    
    # Liberar memória
    spark.stop()
    tf.keras.backend.clear_session()

## Executar Pipeline

Executar o pipeline completo de treino LSTM com configurações apropriadas para a Databricks Community Edition.

In [ ]:
# Configurações recomendadas para Databricks Community Edition (15.3 GB Memory, 2 Cores)
main(
    use_sample=True,           # Usar amostragem para Community Edition
    sample_size_per_class=2000, # 2000 exemplos por classe (ajuste conforme necessário)
    complexity="light",        # Modelo mais leve para recursos limitados
    n_folds=3                  # Menos folds para validação cruzada mais rápida
)

## Configurações Alternativas

Abaixo estão configurações alternativas para diferentes cenários de recursos. Descomente a configuração apropriada para o seu ambiente.

In [ ]:
# # Configuração para recursos moderados (ex: cluster com mais memória)
# main(
#     use_sample=True,
#     sample_size_per_class=5000,  # Amostra maior
#     complexity="standard",       # Modelo padrão
#     n_folds=5                    # Mais folds para validação mais robusta
# )

# # Configuração para recursos abundantes (ex: cluster com GPUs)
# main(
#     use_sample=False,            # Usar dataset completo
#     complexity="deep",           # Modelo mais profundo
#     n_folds=5                    # Validação cruzada completa
# )

## Conclusão

Este notebook demonstrou como implementar, treinar e avaliar um modelo LSTM para deteção de notícias falsas, com considerações específicas para as limitações da Databricks Community Edition.

### Principais pontos:

1. **Amostragem estratificada**: Usada para lidar com limitações de memória, mantendo a distribuição de classes
2. **Validação cruzada**: Implementada para avaliação robusta mesmo com amostras menores
3. **Complexidade ajustável**: Opções para modelos mais leves ou mais profundos dependendo dos recursos disponíveis
4. **Gestão de memória**: Técnicas para liberar memória e evitar problemas de OOM (Out of Memory)

### Próximos passos:

1. Experimentar com diferentes arquiteturas (GRU, Transformers)
2. Implementar técnicas de regularização adicionais
3. Explorar embeddings pré-treinados (GloVe, Word2Vec)
4. Desenvolver um pipeline de inferência para classificação em tempo real